<h1>Import Necessary Libraries</h1>

In [113]:
#1.load data (TODO: set aside image for test and validation)
#2.Define Neural network
#3.Define loss function
#4.Train network on training data
#5.Test network on test data

import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings(action='once')
import torchvision
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os 
from tqdm.notebook import tqdm
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder

In [114]:
#Resizing images
from PIL import Image
import os, sys

#path = r"C:\Users\truon\Desktop\ECE\ECE196\ML project\data\flowers\\"
path = r"/Users/zacharyjohnston/Documents/GitHub/flower/Tai/data/flowers"
dirs = os.listdir(path)

def resize():
    i=0
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((700,700), Image.ANTIALIAS)
            #imResize.save(r"C:\Users\truon\Desktop\ECE\ECE196\ML project\data\subData\imgRe\\" + str(i) + ' resized.jpg', 'JPEG', quality=90)
            imResize.save(r"/Users/zacharyjohnston/Documents/GitHub/flower/Tai/data/subData/imgRe" + str(i) + ' resizedd.jpg', 'JPEG', quality=90)
            i += 1

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
# Load data
#train_dir = r"C:\Users\truon\Desktop\ECE\ECE196\ML project\data\subData"
train_dir = r"/Users/zacharyjohnston/Documents/GitHub/flower/Tai/data/subData"
val_dl = r"/Users/zacharyjohnston/Documents/GitHub/flower/Tai/data/subData/validation_set"

In [119]:
# for image preprocessing
# https://auth0.com/blog/image-processing-in-python-with-pillow/#Resizing-Images
# Resizing images

# image = Image.open('demo_image.jpg')
# new_image = image.resize((400, 400))
# new_image.save('image_400.jpg')

# print(image.size) # Output: (1920, 1280)
# print(new_image.size) # Output: (400, 400)

In [120]:
transform_train = T.Compose([
    T.RandomCrop(128, padding_mode="reflect"),
    T.RandomHorizontalFlip(),
    T.ToTensor()
])
train_ds = ImageFolder(
    root=train_dir,
    transform = transform_train
)

In [121]:
batch_size = 100

In [122]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)

In [123]:
# display image process
def display(train_dl):
    for images,_ in train_dl:
        fig, ax = plt.subplots(figsize=(8,8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:32], nrow=8).permute(1,2,0))
        break

In [124]:
# define model
#class Net(nn.Module)
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda") #if the GPU is availble this method will return cuda.
    else:
        return torch.device("cpu")
    
def to_device(data, device): #in here we move the data to device of our choice, the GPU
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for x in self.dl:
            yield to_device(x, self.device)
            
    def __len__(self):
        return len(self.dl)
    
device = get_device()

In [125]:
def accuracy(out, labels):
    _, preds = torch.max(out, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out =self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {"val_loss": loss.detach(), "val_acc": acc}
    
    def validation_epoch_end(self, outputs):
        batch_loss = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_acc = [x["val_acc"] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {"val_loss": epoch_loss.item(), "val_acc": epoch_acc.item()}
    
    def epoch_end(self, epoch, epochs, result):
        print("Epoch: [{}/{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
        epoch+1, epochs, result["lrs"][-1], result["train_loss"], result["val_loss"], result["val_acc"]))

In [126]:
class ResNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = models.resnet34(pretrained=True)
        number_of_features = self.network.fc.in_features
        self.network.fc = nn.Linear(number_of_features, 104)
        
    def forward(self, xb):
        return self.network(xb)
    
    def freeze(self): #by freezing all the layers but the last one we allow it to warm up (the others are already good at training)
        for param in self.network.parameters():
            param.require_grad=False
        for param in self.network.fc.parameters():
            param.require_grad=True
            
    def unfreeze(self):
        for param in self.network.parameters():
            param.require_grad=True

In [127]:
model = ResNet()
model

ResNet(
  (network): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [128]:
model = to_device(model, device) #let's move the model to the GPU

In [129]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [130]:
@torch.no_grad()
def evaluate(model, val_dl):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_dl]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]
    
def fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, weight_decay=0,
                 grad_clip=None, opt_func=torch.optim.Adam):
    torch.cuda.empty_cache()
    
    history = []
    opt = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr, epochs=epochs,
                                               steps_per_epoch=len(train_dl))
    
    for epoch in range(epochs):
        model.train()
        train_loss = []
        lrs = []
        for batch in tqdm(train_dl):
            loss = model.training_step(batch)
            train_loss.append(loss)
            loss.backward()
            
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            opt.step()
            opt.zero_grad()
            
            lrs.append(get_lr(opt))
            sched.step()
            
        result = evaluate(model, val_dl)
        result["train_loss"] = torch.stack(train_loss).mean().item()
        result["lrs"] = lrs
        model.epoch_end(epoch, epochs, result)
        history.append(result)
    return history

In [131]:
result = evaluate(model, val_dl) #let's check the model performance before training it
result

AttributeError: 'str' object has no attribute 'to'

In [ ]:
model.freeze()

In [ ]:
epochs = 10
max_lr = 10e-4
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
%%time

history = fit_one_cycle(epochs, max_lr, model, train_dl, val_dl,
                       weight_decay=weight_decay, grad_clip=grad_clip,
                       opt_func=opt_func)

In [ ]:
model.unfreeze()

In [ ]:
epochs = 10
max_lr = 0.0005
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
%%time

history = fit_one_cycle(epochs, max_lr, model, train_dl, val_dl,
                       weight_decay=weight_decay, grad_clip=grad_clip,
                       opt_func=opt_func)

In [ ]:
val_loss = [x["val_loss"] for x in history]
train_loss = [x.get("train_loss") for x in history]
plt.plot(val_loss, "-rx")
plt.plot(train_loss, "-gx")
plt.title("Loss vs number of epochs")
plt.legend(["Validation loss", "Train loss"])
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
accuracy = [x["val_acc"] for x in history]
plt.plot(accuracy, "-bx")
plt.title("Acccuracy vs number of epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from torchvision.datasets.folder import default_loader

df = pd.read_csv("../input/flowers/flowers")
df.head()

In [ ]:
class TestData(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.label = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.item()
        
        label = self.label.iloc[idx,1]
        image_path = os.path.join(self.root_dir, f"{self.label.iloc[idx,0]}.jpeg")
        
        image = default_loader(image_path)
        
        if self.transform:
            image = self.transform(image)
        
        return (image, label)

In [ ]:
TEST_DIR = "../input/104-flowers-garden-of-eden/jpeg-224x224/test"

In [ ]:
transform_test = T.Compose([
    T.ToTensor()
])

test_ds = TestData(
    root_dir=TEST_DIR,
    csv_file="../input/flowers/flowers",
    transform=transform_test

)

In [ ]:
def predict_image(image):
    xb = to_device(image.unsqueeze(0), device)
    out = model(xb)
    _, preds = torch.max(out, dim=1)
    prediction = preds[0].item()
    return prediction

In [ ]:
image, label = test_ds[20]
print("Label:", label)
print("Prediction:", predict_image(image))
plt.imshow(image.permute(1,2,0))